
# Results notebook

In this notebook, I have tested out the different algorithms:
- how long they take to find the goal position, for this, start timer and end timer
- memory usage, need to figure this out
- how optimal is the path, this is shown by:
    - distance of the path
    - how many turns need to be taken
    - Using the stats.get_time_to_path(path) func, it can show which path truly takes the least time not depending on distance traveled or number of turns
- To compare between diff algorithms, I need to compare:
    - memory usage by each algorithm (if more than 2gbs, there is a problem)
    - Time it takes for path taken (most important I think)
    - Time taken for algorithm to run
    - Maybe need to run and algorithm on each maze for figuring out the maze, the floodfill x BFS, or just BFS, this will help identify in real life, how long it would take to figure out the maze before the algorithms even start. Most prob would be all offline. If online, we store the path taken by each algorithm, use that same algorithm to get the path back to the start (or use floodfill) while simulatuasly running the algorithm to find the path to get the path to centre. This would give me the time for everything given that I have 10 mins to try taking the most optimal path. ////

To compare your algorithms effectively, especially with a dataset of around 300 mazes from previous competitions, you'll want to consider multiple metrics that highlight different aspects of performance. Here's a structured approach you can use:

### 1. **Metrics Collection**
   - **Memory Usage**: Track the peak and average memory usage during the algorithm's execution. This will help you understand the memory efficiency of each algorithm.
   - **Execution Time**: Measure the total time taken to find a solution for each maze. This includes the entire runtime of the algorithm, from start to finish.
   - **Path Quality Metrics**:
     - **Time to Traverse Path**: Calculate the time the micromouse would take to follow the path, considering acceleration, deceleration, and turns. This helps assess how optimal the path is in practice.
     - **Path Length**: Measure the total distance of the path found by the algorithm. Shorter paths are typically better.
     - **Number of Turns**: Count the number of turns in the path. Fewer turns can indicate a more efficient path in terms of maintaining speed and reducing wear on the mouse.

### 2. **Data Aggregation**
   - **Mean and Median**: Calculate the mean and median of each metric across all 300 mazes. This will give you an idea of the overall performance and consistency of each algorithm.
   - **Standard Deviation**: Assess the variability in performance. High variance might indicate that the algorithm performs well in some cases but poorly in others.
   - **Percentiles**: Look at the 25th, 50th (median), and 75th percentiles to understand the distribution of performance metrics.

### 3. **Comparison Criteria**
   - **Efficiency**: Compare the execution time and memory usage. Algorithms that use less memory and run faster are more efficient.
   - **Path Optimality**: Compare the path quality metrics (time to traverse, path length, number of turns). Algorithms that consistently find shorter, faster paths with fewer turns are more optimal.
   - **Robustness**: Consider how each algorithm handles different maze complexities (e.g., simple vs. complex mazes). An algorithm that performs consistently well across all types of mazes might be more robust.

### 4. **Visualization**
   - **Box Plots**: Use box plots to visualize the distribution of metrics like execution time, path length, and the number of turns. This will help you identify outliers and the general distribution of your data.
   - **Bar Charts**: For comparing mean values across algorithms, bar charts can be useful.
   - **Scatter Plots**: If you want to visualize the relationship between two metrics (e.g., path length vs. number of turns), scatter plots can be insightful.

### 5. **Statistical Analysis**
   - **ANOVA or Kruskal-Wallis Test**: Use these statistical tests to determine if there are significant differences in performance metrics between the algorithms.
   - **Pairwise Comparisons**: If you find significant differences, use pairwise comparison tests (like Tukey's HSD) to determine which algorithms differ significantly from each other.

### 6. **Benchmarking and Case Studies**
   - **Benchmark Mazes**: Identify a few representative mazes from your dataset that are particularly challenging or typical. Compare how each algorithm performs on these mazes in detail.
   - **Case Studies**: Perform a detailed analysis of a few selected mazes where algorithms show distinct performance differences. This can provide insights into why certain algorithms perform better or worse in specific scenarios.

### 7. **Summary and Recommendations**
   - **Rank the Algorithms**: Based on the collected data, rank the algorithms according to different criteria (e.g., overall efficiency, path optimality).
   - **Identify Trade-offs**: Highlight any trade-offs between different algorithms (e.g., one might be faster but produce longer paths).
   - **Make Recommendations**: Based on your findings, recommend which algorithm might be best suited for different types of mazes or competition settings.

This comprehensive approach will allow you to make well-informed decisions about which algorithm is the best overall and which might be better suited to specific scenarios.

In [1]:
import glob
import json
import os

from algorithms.utilities.Utils import Utils
from algorithms.utilities.Stats import Stats

from algorithms.classical.floodfill.FloodFill import FloodFill
from algorithms.classical.bfs.BFS import BFS
from algorithms.classical.dijkstra.Dijkstra import Dijkstra
from algorithms.classical.astar.AStar import AStar

from algorithms.reinforcement_learning.sarsa.Sarsa import Sarsa
from algorithms.reinforcement_learning.q_learning.Qlearning import QLearning
from algorithms.reinforcement_learning.dynaq.DynaQLearning import DynaQLearning
from algorithms.reinforcement_learning.dynaq.DynaQSarsa import DynaQSarsa

from algorithms.ExploreMaze import ExploreMaze
import pandas as pd


C:\Users\ACER\AppData\Local\Temp\ipykernel_4608\3884277656.py:19: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
maze = Utils.load_maze('mazes/competition_json/alljapan-031-2010-exp-fin.json')

In [3]:
def get_mazes_json(dirname):
    mazes = {}
    for maze_file in glob.glob(os.path.join(dirname, '*.json')):
        with open(maze_file, 'r') as f:
            maze_name = maze_file.split('\\')[1]
            maze_name = maze_name.split('.')[0]
            mazes[maze_name] = {eval(k): v for k, v in json.load(f).items()} 
    return mazes

In [4]:
mazes = get_mazes_json('mazes/competition_json')

In [5]:
import time
def get_runtime(start_time):
    end_time = time.perf_counter_ns()
    return end_time - start_time

In [6]:
remove_mazes = []
for name, item in mazes.items():
    try:
        flood = FloodFill(walls=item)
        path = flood.get_path_from_flood_map()
    except Exception as e:
        remove_mazes.append(name)

for name in remove_mazes:
    del mazes[name]

## Classical Algorithms

In [7]:
# utils = Utils()
# stats = Stats()
# explore = ExploreMaze(maze=maze)
# explore.move_and_floodfill()
# walls = explore.get_walls()
# utils.draw_maze(walls, dist_map=explore.flood_fill(explore.get_goal_position()), path=explore.path)
# # print(walls == maze)
# print(explore.path)
# print(stats.get_time_from_path(explore.path))

### Floodfill

In [8]:
stats = Stats()
flood_paths = []
flood_path_times = []
flood_turns = []
flood_distances = []
flood_cells_travelled = []
flood_exec_time = []
flood_memory = []
for name, item in mazes.items():
    start_time = time.perf_counter_ns()
    flood = FloodFill(walls=item)
    flood_exec_time.append(get_runtime(start_time) * pow(10, -3))
    path = flood.get_path_from_flood_map()
    flood_paths.append(path)
    flood_path_times.append(stats.get_time_from_path(path))
    dist, turns = flood.get_stats()
    flood_cells_travelled.append(dist)
    flood_distances.append(stats.get_dist_travelled())
    flood_turns.append(turns)
    flood_memory.append(flood.total_memory_used)

maze_names = mazes.keys()

data = {
    'Maze Name': maze_names,
    'Path': flood_paths,
    'Path Time (s)': flood_path_times,
    'Turns': flood_turns,
    'Distance Travelled (m)': flood_distances,
    'Execution Time (micro secs)': flood_exec_time,
    'Memory Usage (MB)': flood_memory,
    'Cells Travelled': flood_cells_travelled
}

flood_fill_df = pd.DataFrame(data).set_index('Maze Name')
flood_fill_df.to_csv('results/csv_files/competition/flood_fill.csv')

In [9]:
flood_fill_df

,Path,Path Time (s),Turns,Distance Travelled (m),Execution Time (micro secs),Memory Usage (MB),Cells Travelled
Maze Name,,,,,,,
13ye,"[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5...",18.241782,21,11.97,112.8,0.000000,68
50,"[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5...",27.575872,38,11.25,115.2,0.000000,64
86,"[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5...",5.598013,6,3.69,132.9,0.000000,25
87sin,"[(0, 0), (0, 1), (0, 2), (0, 3), (1, 3), (1, 2...",13.623074,15,8.55,86.6,0.000000,49
87us1,"[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5...",24.619658,32,11.25,81.1,0.000000,65
...,...,...,...,...,...,...,...
xx,"[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5...",9.109604,9,6.75,88.0,0.003906,40
yama2002,"[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5...",14.739266,16,9.63,209.1,0.007812,55
yama7,"[(0, 0), (0, 1), (0, 2), (1, 2), (2, 2), (3, 2...",12.889292,14,8.55,80.0,0.003906,54


In [10]:
# flood = FloodFill(walls=maze)
# flood_path = flood.get_path_from_flood_map()
# print(flood.get_stats())

In [11]:
# Utils().draw_maze(walls=maze, path=flood_path, dist_map= flood.flood_map)

In [12]:
# stats = Stats()
# print(stats.get_time_from_path(flood_path))

### BFS

In [13]:
stats = Stats()
bfs_paths = []
bfs_path_times = []
bfs_turns = []
bfs_distances = []
bfs_cells_travelled = []
bfs_exec_time = []
bfs_memory = []
for name, item in mazes.items():
    start_time = time.perf_counter_ns()
    bfs = BFS(walls=item)
    bfs_exec_time.append(get_runtime(start_time) * pow(10, -3))
    path = bfs.find_shortest_path_to_goal()
    bfs_paths.append(path)
    bfs_path_times.append(stats.get_time_from_path(path))
    dist, turns = bfs.get_stats()
    bfs_cells_travelled.append(dist)
    bfs_distances.append(stats.get_dist_travelled())
    bfs_turns.append(turns)
    bfs_memory.append(bfs.total_memory_used)

maze_names = mazes.keys()

data = {
    'Maze Name': maze_names,
    'Path': bfs_paths,
    'Path Time (s)': bfs_path_times,
    'Turns': bfs_turns,
    'Distance Travelled (m)': bfs_distances,
    'Execution Time (micro secs)': bfs_exec_time,
    'Memory Usage (MB)': bfs_memory,
    'Cells Travelled': bfs_cells_travelled
}

bfs_df = pd.DataFrame(data).set_index('Maze Name')
bfs_df.to_csv('results/csv_files/competition/bfs.csv')

In [14]:
bfs_df

,Path,Path Time (s),Turns,Distance Travelled (m),Execution Time (micro secs),Memory Usage (MB),Cells Travelled
Maze Name,,,,,,,
13ye,"[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5...",19.481086,23,11.97,116.1,0.000000,68
50,"[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5...",27.575872,38,11.25,108.0,0.000000,64
86,"[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5...",5.598013,6,3.69,85.1,0.000000,25
87sin,"[(0, 0), (0, 1), (0, 2), (0, 3), (1, 3), (1, 2...",14.794908,17,8.55,84.2,0.003906,49
87us1,"[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5...",24.619658,32,11.25,90.8,0.000000,65
...,...,...,...,...,...,...,...
xx,"[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5...",11.547772,13,6.75,67.2,0.000000,40
yama2002,"[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5...",14.739266,16,9.63,100.2,0.000000,55
yama7,"[(0, 0), (0, 1), (0, 2), (1, 2), (2, 2), (3, 2...",12.889292,14,8.55,69.2,0.000000,54


In [15]:
# bfs = BFS(walls=maze)
# bfs_path = bfs.find_shortest_path_to_goal()
# print(bfs.get_stats())

In [16]:
# Utils().draw_maze(walls=maze, path=bfs_path, dist_map = bfs.distances)

In [17]:
# stats = Stats()
# print(stats.get_time_from_path(bfs_path))

### Dijkstra

In [18]:
stats = Stats()
dijkstra_paths = []
dijkstra_path_times = []
dijkstra_turns = []
dijkstra_distances = []
dijkstra_cells_travelled = []
dijkstra_exec_time = []
dijkstra_memory = []
for name, item in mazes.items():
    start_time = time.perf_counter_ns()
    dijkstra = Dijkstra(walls=item)
    dijkstra_exec_time.append(get_runtime(start_time) * pow(10, -3))
    path = dijkstra.find_shortest_path_to_goal()
    dijkstra_paths.append(path)
    dijkstra_path_times.append(stats.get_time_from_path(path))
    dist, turns = dijkstra.get_stats()
    dijkstra_cells_travelled.append(dist)
    dijkstra_distances.append(stats.get_dist_travelled())
    dijkstra_turns.append(turns)
    dijkstra_memory.append(dijkstra.total_memory_used)

maze_names = mazes.keys()

data = {
    'Maze Name': maze_names,
    'Path': dijkstra_paths,
    'Path Time (s)': dijkstra_path_times,
    'Turns': dijkstra_turns,
    'Distance Travelled (m)': dijkstra_distances,
    'Execution Time (micro secs)': dijkstra_exec_time,
    'Memory Usage (MB)': dijkstra_memory,
    'Cells Travelled': dijkstra_cells_travelled
}

dijkstra_df = pd.DataFrame(data).set_index('Maze Name')
dijkstra_df.to_csv('results/csv_files/competition/dijkstra.csv')

In [19]:
dijkstra_df

,Path,Path Time (s),Turns,Distance Travelled (m),Execution Time (micro secs),Memory Usage (MB),Cells Travelled
Maze Name,,,,,,,
13ye,"[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5...",19.481086,24,11.97,156.4,0.0,68
50,"[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5...",27.575872,38,11.25,132.2,0.0,64
86,"[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5...",5.598013,8,3.69,184.6,0.0,25
87sin,"[(0, 0), (0, 1), (0, 2), (0, 3), (1, 3), (1, 2...",14.794908,18,8.55,83.7,0.0,49
87us1,"[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5...",24.619658,34,11.25,76.4,0.0,65
...,...,...,...,...,...,...,...
xx,"[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5...",11.547772,14,6.75,89.3,0.0,40
yama2002,"[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5...",14.739266,18,9.63,71.9,0.0,55
yama7,"[(0, 0), (0, 1), (0, 2), (1, 2), (2, 2), (3, 2...",12.889292,14,8.55,70.4,0.0,54


In [20]:
# dijkstra = Dijkstra(walls=maze)
# dijkstra_path = dijkstra.find_shortest_path_to_goal()
# print(dijkstra.get_stats())

In [21]:
# Utils().draw_maze(walls=maze, path=dijkstra_path, dist_map = dijkstra.distances)

In [22]:
# print(stats.get_time_from_path(dijkstra_path))

### A*

In [23]:
stats = Stats()
astar_paths = []
astar_path_times = []
astar_turns = []
astar_distances = []
astar_cells_travelled = []
astar_exec_time = []
astar_memory = []
for name, item in mazes.items():
    start_time = time.perf_counter_ns()
    astar = AStar(walls=item)
    astar_exec_time.append(get_runtime(start_time) * pow(10, -3))
    path = astar.find_shortest_path_to_goal()
    astar_paths.append(path)
    astar_path_times.append(stats.get_time_from_path(path))
    dist, turns = astar.get_stats()
    astar_cells_travelled.append(dist)
    astar_distances.append(stats.get_dist_travelled())
    astar_turns.append(turns)
    astar_memory.append(astar.total_memory_used)

maze_names = mazes.keys()

data = {
    'Maze Name': maze_names,
    'Path': astar_paths,
    'Path Time (s)': astar_path_times,
    'Turns': astar_turns,
    'Distance Travelled (m)': astar_distances,
    'Execution Time (micro secs)': astar_exec_time,
    'Memory Usage (MB)': astar_memory,
    'Cells Travelled': astar_cells_travelled
}

astar_df = pd.DataFrame(data).set_index('Maze Name')
astar_df.to_csv('results/csv_files/competition/astar.csv')

In [24]:
astar_df

,Path,Path Time (s),Turns,Distance Travelled (m),Execution Time (micro secs),Memory Usage (MB),Cells Travelled
Maze Name,,,,,,,
13ye,"[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5...",19.481086,24,11.97,114.4,0.0,68
50,"[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5...",27.575872,38,11.25,119.6,0.0,64
86,"[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5...",5.598013,8,3.69,100.5,0.0,25
87sin,"[(0, 0), (0, 1), (0, 2), (0, 3), (1, 3), (1, 2...",14.794908,18,8.55,103.0,0.0,49
87us1,"[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5...",24.619658,34,11.25,104.3,0.0,65
...,...,...,...,...,...,...,...
xx,"[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5...",11.547772,14,6.75,73.2,0.0,40
yama2002,"[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5...",14.739266,18,9.63,71.6,0.0,55
yama7,"[(0, 0), (0, 1), (0, 2), (1, 2), (2, 2), (3, 2...",12.889292,14,8.55,73.1,0.0,54


In [25]:
# astar = AStar(walls=maze)
# astar_path = (astar.find_shortest_path_to_goal())
# print(astar.get_stats())

In [26]:
# Utils().draw_maze(walls=maze, path=astar_path, dist_map = astar.distances)

In [27]:
# stats = Stats()
# print(stats.get_time_from_path(astar_path))

## Reinforcememnt Learning

### Sarsa

In [30]:
stats = Stats()
sarsa_paths = []
sarsa_path_times = []
sarsa_turns = []
sarsa_distances = []
sarsa_cells_travelled = []
sarsa_exec_time = []
sarsa_memory = []
for name, item in mazes.items():
    print(f'running {name}')
    start_time = time.perf_counter_ns()
    sarsa = Sarsa(walls=item)
    sarsa.run_sarsa()
    sarsa_exec_time.append(get_runtime(start_time) * pow(10, -3))
    path = sarsa.get_path()
    sarsa_paths.append(path)
    sarsa_path_times.append(stats.get_time_from_path(path))
    dist, turns = sarsa.get_stats()
    sarsa_cells_travelled.append(dist)
    sarsa_distances.append(stats.get_dist_travelled())
    sarsa_turns.append(turns)
    sarsa_memory.append(sarsa.total_memory_used)

maze_names = mazes.keys()

data = {
    'Maze Name': maze_names,
    'Path': sarsa_paths,
    'Path Time (s)': sarsa_path_times,
    'Turns': sarsa_turns,
    'Distance Travelled (m)': sarsa_distances,
    'Execution Time (micro secs)': sarsa_exec_time,
    'Memory Usage (MB)': sarsa_memory,
    'Cells Travelled': sarsa_cells_travelled
}

sarsa_df = pd.DataFrame(data).set_index('Maze Name')
sarsa_df.to_csv('results/csv_files/competition/sarsa_using_time.csv')

running 13ye
running agent 1
running agent 2
running agent 3
running 50
running agent 1
running agent 2
running agent 3
running 86
running agent 1
running agent 2
running agent 3
running 87sin
running agent 1
running agent 2
running agent 3
running 87us1
running agent 1
running agent 2
running agent 3
running 88
running agent 1
running agent 2
running agent 3
running a
running agent 1
running agent 2
running agent 3
running AAMC15Maze
running agent 1
running agent 2
running agent 3
running AAMC16Maze
running agent 1
running agent 2
running agent 3
running AAMC17Maze
running agent 1
running agent 2
running agent 3
running AAMC18Maze
running agent 1
running agent 2
running agent 3
running AAMC22Maze
running agent 1
running agent 2
running agent 3
running AAMC23Maze
running agent 1
running agent 2
running agent 3
running AAMC24Maze
running agent 1
running agent 2
running agent 3
running AAMCUCLAMM2018
running agent 1
running agent 2
running agent 3
running allamerica2013
running agent 1
r

KeyboardInterrupt: 

In [ ]:
sarsa = Sarsa(walls=maze)

In [ ]:
sarsa.run_sarsa()
sarsa_path = sarsa.get_path()

In [ ]:
Utils.draw_maze(walls=maze, dist_map=sarsa.get_max_q_values(), path=sarsa_path)

In [ ]:
# stats = Stats()
# stats.get_time_from_path(sarsa_path)

### QLearning


In [ ]:
stats = Stats()
qlearning_paths = []
qlearning_path_times = []
qlearning_turns = []
qlearning_distances = []
qlearning_cells_travelled = []
qlearning_exec_time = []
qlearning_memory = []
for name, item in mazes.items():
    print(f'running {name}')
    start_time = time.perf_counter_ns()
    qlearning = QLearning(walls=item)
    qlearning.run_qlearning()
    qlearning_exec_time.append(get_runtime(start_time) * pow(10, -3))
    path = qlearning.get_path()
    qlearning_paths.append(path)
    qlearning_path_times.append(stats.get_time_from_path(path))
    dist, turns = qlearning.get_stats()
    qlearning_cells_travelled.append(dist)
    qlearning_distances.append(stats.get_dist_travelled())
    qlearning_turns.append(turns)
    qlearning_memory.append(qlearning.total_memory_used)

maze_names = mazes.keys()

data = {
    'Maze Name': maze_names,
    'Path': qlearning_paths,
    'Path Time (s)': qlearning_path_times,
    'Turns': qlearning_turns,
    'Distance Travelled (m)': qlearning_distances,
    'Execution Time (micro secs)': qlearning_exec_time,
    'Memory Usage (MB)': qlearning_memory,
    'Cells Travelled': qlearning_cells_travelled
}

qlearning_df = pd.DataFrame(data).set_index('Maze Name')
qlearning_df.to_csv('results/csv_files/competition/qlearning_using_timw.csv')

In [ ]:
# qlearning = QLearning(walls=maze)
# qlearning.run_qlearning()
# qlearning_path = qlearning.get_path()

In [ ]:
# Utils.draw_maze(walls=maze, dist_map=qlearning.get_max_q_values(), path=qlearning_path)


In [ ]:
# stats = Stats()
# stats.get_time_from_path(qlearning_path)

### DynaQ QLearning

In [ ]:
# from algorithms.reinforcement_learning.dynaq.DynaQLearning import DynaQLearning, Utils
# maze = Utils.load_maze('mazes/competition_json/alljapan-006-1985-pre.json')
# dyna_qlearning = DynaQLearning(walls=maze)
# dyna_qlearning.run_dyna_qlearning()
# # print(dyna_qlearning.unfeasable_paths)
# Utils.draw_maze(maze, unfeasable=dyna_qlearning.unfeasable_paths, path=dyna_qlearning.get_path())
# print(dyna_qlearning.max_episodes)

In [ ]:
from algorithms.reinforcement_learning.dynaq.DynaQLearning import DynaQLearning, Utils
from algorithms.utilities.Stats import Stats
stats = Stats()
dyna_qlearning_paths = []
dyna_qlearning_path_times = []
dyna_qlearning_turns = []
dyna_qlearning_distances = []
dyna_qlearning_cells_travelled = []
dyna_qlearning_exec_time = []
dyna_qlearning_memory = []
for name, item in mazes.items():
    print(f'\nrunning {name}')
    start_time = time.perf_counter_ns()
    dyna_qlearning = DynaQLearning(walls=item)
    dyna_qlearning.run_dyna_qlearning()
    dyna_qlearning_exec_time.append(get_runtime(start_time) * pow(10, -3))
    path = dyna_qlearning.get_path()
    dyna_qlearning_paths.append(path)
    dyna_qlearning_path_times.append(stats.get_time_from_path(path))
    dist, turns = dyna_qlearning.get_stats()
    dyna_qlearning_cells_travelled.append(dist)
    dyna_qlearning_distances.append(stats.get_dist_travelled())
    dyna_qlearning_turns.append(turns)
    dyna_qlearning_memory.append(dyna_qlearning.total_memory_used)

maze_names = mazes.keys()

data = {
    'Maze Name': maze_names,
    'Path': dyna_qlearning_paths,
    'Path Time (s)': dyna_qlearning_path_times,
    'Turns': dyna_qlearning_turns,
    'Distance Travelled (m)': dyna_qlearning_distances,
    'Execution Time (micro secs)': dyna_qlearning_exec_time,
    'Memory Usage (MB)': dyna_qlearning_memory,
    'Cells Travelled': dyna_qlearning_cells_travelled,
}

dyna_qlearning_df = pd.DataFrame(data).set_index('Maze Name')
dyna_qlearning_df.to_csv('results/csv_files/competition/dyna_qlearning_using_time.csv')

In [ ]:
# maze_names = mazes.keys()
# 
# data = {
#     'Maze Name': maze_names,
#     'Path': dyna_qlearning_paths,
#     'Path Time (s)': dyna_qlearning_path_times,
#     'Turns': dyna_qlearning_turns,
#     'Distance Travelled (m)': dyna_qlearning_distances,
#     'Execution Time (micro secs)': dyna_qlearning_exec_time,
#     'Memory Usage (MB)': dyna_qlearning_memory,
#     'Cells Travelled': dyna_qlearning_cells_travelled,
# }
# 
# dyna_qlearning_df = pd.DataFrame(data).set_index('Maze Name')

In [ ]:
# faield = dyna_qlearning_df[dyna_qlearning_df['Failed To Use Equations of Motion'] == True]

In [ ]:
# faield

In [ ]:
# dyna_qlearning = DynaQLearning(walls=maze)
# dyna_qlearning.run_dyna_qlearning()
# dyna_qlearning_path = dyna_qlearning.get_path()

In [ ]:
# Utils.draw_maze(walls=maze, dist_map=dyna_qlearning.get_max_q_values(), path=dyna_qlearning_path)

In [ ]:
# stats = Stats()
# stats.get_time_from_path(dyna_qlearning_path)

In [ ]:
# from algorithms.reinforcement_learning.dynaq.DynaQLearning import DynaQLearning, Utils
# from algorithms.utilities.Stats import Stats
# stats = Stats()
# dyna_qlearning_paths_2 = []
# dyna_qlearning_path_times_2 = []
# dyna_qlearning_turns_2 = []
# dyna_qlearning_distances_2 = []
# dyna_qlearning_cells_travelled_2 = []
# dyna_qlearning_exec_time_2 = []
# dyna_qlearning_memory_2 = []
# dyna_qlearning_failed_2 = []
# for name, item in mazes.items():
#     print(f'\nrunning {name}')
#     start_time = time.perf_counter_ns()
#     dyna_qlearning = DynaQLearning(walls=item)
#     dyna_qlearning.run_dyna_qlearning()
#     dyna_qlearning_exec_time_2.append(get_runtime(start_time) * pow(10, -3))
#     path = dyna_qlearning.get_path()
#     dyna_qlearning_paths_2.append(path)
#     dyna_qlearning_path_times_2.append(stats.get_time_from_path(path))
#     dist, turns = dyna_qlearning.get_stats()
#     dyna_qlearning_cells_travelled_2.append(dist)
#     dyna_qlearning_distances_2.append(stats.get_dist_travelled())
#     dyna_qlearning_turns_2.append(turns)
#     dyna_qlearning_memory_2.append(dyna_qlearning.total_memory_used)
#     dyna_qlearning_failed_2.append(dyna_qlearning.fails)
# 
# maze_names = mazes.keys()
# 
# data = {
#     'Maze Name': maze_names,
#     'Path': dyna_qlearning_paths_2,
#     'Path Time (s)': dyna_qlearning_path_times_2,
#     'Turns': dyna_qlearning_turns_2,
#     'Distance Travelled (m)': dyna_qlearning_distances_2,
#     'Execution Time (micro secs)': dyna_qlearning_exec_time_2,
#     'Memory Usage (MB)': dyna_qlearning_memory_2,
#     'Cells Travelled': dyna_qlearning_cells_travelled_2,
#     'Failed To Use Equations of Motion': dyna_qlearning_failed_2
# }
# 
# dyna_qlearning_df_2 = pd.DataFrame(data).set_index('Maze Name')
# dyna_qlearning_df_2.to_csv('results/csv_files/competition/dyna_qlearning_2.csv')

In [ ]:
# dyna_qlearning_df_2 = pd.DataFrame(data).set_index('Maze Name')
# dyna_qlearning_df_2.to_csv('results/csv_files/competition/dyna_qlearning_2.csv')

In [ ]:
# dyna_qlearning_df_2

In [ ]:
# failed_2 = dyna_qlearning_df_2[dyna_qlearning_df_2['Failed To Use Equations of Motion'] == True]

In [ ]:
# failed_2


In [ ]:
# failed_2.index


### DynaQ Sarsa


In [ ]:
stats = Stats()
dynaq_sarsa_paths = []
dynaq_sarsa_path_times = []
dynaq_sarsa_turns = []
dynaq_sarsa_distances = []
dynaq_sarsa_cells_travelled = []
dynaq_sarsa_exec_time = []
dynaq_sarsa_memory = []
for name, item in mazes.items():
    print(f'running {name}')
    start_time = time.perf_counter_ns()
    dynaq_sarsa = DynaQSarsa(walls=item)
    dynaq_sarsa.run_dynaq_sarsa()
    dynaq_sarsa_exec_time.append(get_runtime(start_time) * pow(10, -3))
    path = dynaq_sarsa.get_path()
    dynaq_sarsa_paths.append(path)
    dynaq_sarsa_path_times.append(stats.get_time_from_path(path))
    dist, turns = dynaq_sarsa.get_stats()
    dynaq_sarsa_cells_travelled.append(dist)
    dynaq_sarsa_distances.append(stats.get_dist_travelled())
    dynaq_sarsa_turns.append(turns)
    dynaq_sarsa_memory.append(dynaq_sarsa.total_memory_used)

maze_names = mazes.keys()

data = {
    'Maze Name': maze_names,
    'Path': dynaq_sarsa_paths,
    'Path Time (s)': dynaq_sarsa_path_times,
    'Turns': dynaq_sarsa_turns,
    'Distance Travelled (m)': dynaq_sarsa_distances,
    'Execution Time (micro secs)': dynaq_sarsa_exec_time,
    'Memory Usage (MB)': dynaq_sarsa_memory,
    'Cells Travelled': dynaq_sarsa_cells_travelled
}

dynaq_sarsa_df = pd.DataFrame(data).set_index('Maze Name')
dynaq_sarsa_df.to_csv('results/csv_files/competition/dynaq_sarsa_using_time.csv')

In [ ]:
dynaq_sarsa = DynaQSarsa(walls=maze)
dynaq_sarsa.run_dyna_sarsa()
dynaq_sarsa_path = dynaq_sarsa.get_path()

In [ ]:
Utils.draw_maze(walls=maze, dist_map=dynaq_sarsa.get_max_q_values(), path=dynaq_sarsa_path)

In [ ]:
stats = Stats()
stats.get_time_from_path(dynaq_sarsa_path)

In [ ]:
# Max deceleration on micro-mouse when calculating reward
1/(2*0.09)